In [53]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [54]:
import trafficDataUtility as util

In [55]:
path = "C:\\Users\\KangYounKook\\Desktop\\ML_Study\\"

In [56]:
dataset = {}
dataset = util.getDatasetFromFile(path+"sectionInfo.txt")

getDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\sectionInfo.txt


In [57]:
filename = "PeriodTotalAverage.txt"
util.updateDatasetFromFile(path+filename, dataset)

loadDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\PeriodTotalAverage.txt
update Dataset [move_count,work_count,ratio]


In [117]:
classes = 10

In [163]:
filename = "input10_all10.csv"
#filename = "input" + str(classes) +"_all.csv"
#filename = "input10_2min.csv"
data = util.loadCsvInput(path,filename)

loadCsvInput  C:\Users\KangYounKook\Desktop\ML_Study\input10_all.csv


In [164]:
#selectedList = dataset.keys()
selectedList = ['Section697','Section754','Section1212','Section852']
#print(dataset['Section697'])

In [149]:
def getFeedXY(Xa,Ya,k,count):
    Yn = Ya[k:count]
    Xn = Xa[:count-k]
    for i in range(1,k):
        Xn = np.c_[Xn, Xa[i:count-k+i]]
    
    m = int((count-k)*0.7)
    Xt = Xn[0:m]
    Xval = Xn[m:count-k]
    Yt = Yn[0:m]
    Yval = Yn[m:count-k]

    return Xt,Xval,Yt,Yval

In [152]:
def getFeedXY_norm(Xa,Ya,k,count,z):
    Yn = Ya[k:count]/z
    Yn = np.round(Yn)
    Xn = Xa[:count-k]
    for i in range(1,k):
        Xn = np.c_[Xn, Xa[i:count-k+i]]
    
    m = int((count-k)*0.7)
    Xt = Xn[0:m]
    Xval = Xn[m:count-k]
    Yt = Yn[0:m]
    Yval = Yn[m:count-k]

    return Xt,Xval,Yt,Yval

In [153]:
#cnnModel(data,dataset,classes,selectedList)

In [122]:
def cnnModel2(data,dataset,classes,selectedList):
    outfilename = "CNN_Result" + str(classes) + ".txt"
    f = open(outfilename,'w')

    num = 0
    for id in selectedList:
        if id not in data.columns:
            continue
        if dataset[id][5] < 144:
            continue
        if dataset[id][7] > 80:
            continue
        
        Xa = util.getX(id,data,dataset)
        if Xa is None:
            continue
        Ya = np.array(data[id]).reshape(-1,1)

        num += 1
        print(num, id, '-------------------------')
        timeLen = 5
        Xt,Xval,Yt,Yval = getFeedXY(Xa,Ya,timeLen,1440)

        input_size = 5*timeLen
        
        X,Y,Y_one_hot,hypothesis = tf_model(timeLen)    
    
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y_one_hot))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
        prediction = tf.argmax(hypothesis, 1)
        correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        for epoch in range(200):
            c, _ = sess.run([cost,optimizer], feed_dict={X:Xt,Y:Yt})
    
        trainAccuracy = sess.run(accuracy, feed_dict={X:Xt,Y:Yt})
        valAccuracy = sess.run(accuracy, feed_dict={X:Xval,Y:Yval})    
        print('Training:', trainAccuracy)
        print('Validation:', valAccuracy)

        log = id
        log += ',' + dataset[id][0]
        log += ',' + dataset[id][1]
        log += ',' + str(dataset[id][5])
        log += ',' + str(dataset[id][7])
        log += ',' + str(trainAccuracy)
        log += ',' + str(valAccuracy)
        log += '\n'
        f.write(log)
    f.close()

In [202]:
def cnnModel3(data,dataset,classes,selectedList):
    outfilename = "CNN_Result" + str(classes) + ".txt"
    f = open(outfilename,'w')

    num = 0
    for id in selectedList:
        if id not in data.columns:
            continue
        if dataset[id][5] < 144:
            continue
        if dataset[id][7] > 80:
            continue
        
        Xa = util.getX(id,data,dataset)
        if Xa is None:
            continue
        Ya = np.array(data[id]).reshape(-1,1)

        num += 1
        print(num, id, '-------------------------')
        timeLen = 5
        Xt,Xval,Yt,Yval = getFeedXY_norm(Xa,Ya,timeLen,1440,1)
        print(Xt.shape)
        print(Yval.shape)
        print(Yt.shape)
        print(Yval.shape)

        input_size = 5*timeLen
        
        X,Y,Y_one_hot,hypothesis = tf_model2(timeLen,classes)    
    
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y_one_hot))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
        prediction = tf.argmax(hypothesis, 1)
        correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        for epoch in range(200):
            c, _ = sess.run([cost,optimizer], feed_dict={X:Xt,Y:Yt})
    
        trainAccuracy = sess.run(accuracy, feed_dict={X:Xt,Y:Yt})
        valAccuracy = sess.run(accuracy, feed_dict={X:Xval,Y:Yval})    
        print('Training:', trainAccuracy)
        print('Validation:', valAccuracy)

        print(sess.run(prediction, feed_dict={X:Xval,Y:Yval}))
        
        log = id
        log += ',' + dataset[id][0]
        log += ',' + dataset[id][1]
        log += ',' + str(dataset[id][5])
        log += ',' + str(dataset[id][7])
        log += ',' + str(trainAccuracy)
        log += ',' + str(valAccuracy)
        log += '\n'
        f.write(log)
    f.close()

In [199]:
def tf_model(timeLen):
    
    X = tf.placeholder(tf.float32, [None, input_size])
    X_img = tf.reshape(X, [-1, 5, timeLen, 1])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W1 = tf.Variable(tf.random_normal([2,2,1,10], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
    L1 = tf.nn.relu(L1)

    W2 = tf.Variable(tf.random_normal([3,3,10,10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)

    L3 = tf.reshape(L2, [-1, 3*10])

    W3 = tf.Variable(tf.random_normal([3*10, classes], stddev=0.01))
    #W5 = tf.get_variable("W5", shape=[30, classes], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([classes]))

    hypothesis = tf.matmul(L3,W3) + b3
    return X,Y,Y_one_hot,hypothesis


In [200]:
def tf_model2(timeLen,classes):
    
    X = tf.placeholder(tf.float32, [None, 5*timeLen], name='x')
    X_img = tf.reshape(X, [-1, 5, timeLen, 1])
    Y = tf.placeholder(tf.int32, [None, 1], name='y')
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W1 = tf.Variable(tf.random_normal([2,2,1,10], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
    L1 = tf.nn.relu(L1)

    W2 = tf.Variable(tf.random_normal([3,3,10,10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)

    W3 = tf.Variable(tf.random_normal([2,2,10,10], stddev=0.01))
    L3 = tf.nn.conv2d(L2, W2, strides=[1,1,1,1], padding='VALID')
    L3 = tf.nn.relu(L3)

    L4 = tf.reshape(L3, [-1, 4*10])

    W4 = tf.Variable(tf.random_normal([4*10, classes], stddev=0.01))
    #W5 = tf.get_variable("W5", shape=[30, classes], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([classes]))

    hypothesis = tf.matmul(L4,W4) + b4
    return X,Y,Y_one_hot,hypothesis


In [203]:
#cnnModel2(data,dataset,classes,selectedList)
cnnModel3(data,dataset,classes,selectedList)

1 Section697 -------------------------
(1004, 25)
(431, 1)
(1004, 1)
(431, 1)


InvalidArgumentError: logits and labels must be same size: logits_size=[251,10] labels_size=[1004,10]
	 [[Node: SoftmaxCrossEntropyWithLogits_190 = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape_1164, Reshape_1165)]]

Caused by op 'SoftmaxCrossEntropyWithLogits_190', defined at:
  File "C:\Users\KangYounKook\Anaconda3\envs\py35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\KangYounKook\Anaconda3\envs\py35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\KangYounKook\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\KangYounKook\Anaconda3\envs\py35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\KangYounKook\Anaconda3\envs\py35\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance(